In [1]:
import torch


def conjunctin_mul(X, W):
    X_neg = 1 - X
    full_X = torch.cat((X, X_neg), dim=1)
    matrix_X = full_X.repeat(W.shape[0], 1)
    mask = W > 0
    masked_X = torch.where(mask, matrix_X, torch.tensor(1))
    result = torch.prod(masked_X, dim=1, keepdim=True).view(1,-1)
    return result



class TsetlinLayer:
    
    def __init__(self, in_size, out_size):
        w_pos = torch.randint(0, 2, (out_size, in_size,))
        w_neg = 1 - w_pos
        self.w = torch.cat((w_pos, w_neg), dim=1)

    def forward(self, X):
        



    

In [35]:
import torch

w_pos = torch.randint(0, 2, (5, 3,))
w_neg = 1 - w_pos
print(w_neg)
w = torch.cat((w_pos, w_neg), dim=1)
w


tensor([[1, 0, 0],
        [0, 0, 1],
        [1, 0, 1],
        [0, 1, 0],
        [1, 1, 1]])


tensor([[0, 1, 1, 1, 0, 0],
        [1, 1, 0, 0, 0, 1],
        [0, 1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 1, 1, 1]])